In [1]:
import numpy as np
import itertools
import sys
if "../" not in sys.path:
  sys.path.append("../") 
from lib.utils import read_data_from_file, sign

In [2]:
def weighted_pocket_pla(X, y, updates=200, weight=[1000, 1]):
    """
    Weighted Pocket Perceptron Learning Algorithm
    Args:
        X: 数据
        Y: 标签
        updates: 更新次数
        weight: weight[0]->FP weight[1]->FN
    
    Returns:
        w_pocket: 最优特征权重
        w: 最后更新得到的特征权重
    """
    
    def cal_prob(mistakes):
        if len(mistakes) == 0:
            return 0, 0, 0
        
        # cal prob distribution
        FP_index = np.where(y[mistakes]==-1)[0]
        FN_index = np.where(y[mistakes]== 1)[0]
        prob = np.ones_like(mistakes) / (len(FP_index)*weight[0] + len(FN_index)*weight[1]) 
        prob[FP_index] *= weight[0]
        prob[FN_index] *= weight[1]
        np.testing.assert_array_almost_equal(np.sum(prob), 1.0, decimal=5)
        return len(FP_index), len(FN_index), prob
    
    N = len(X)
    w = np.zeros_like(X[0])
    w_pocket = w
    mistakes = np.where(sign(X.dot(w)) != y)[0]         # get index of mistakes
    
    FP, FN, prob = cal_prob(mistakes)                   # cal weight info
    mis_pocket = FP*weight[0] + FN*weight[1]
    
    for i in range(updates):
        mistake = np.random.choice(mistakes, p=prob)            # pike up one mistake randomly
        w = w + X[mistake]*y[mistake]
        mistakes = np.where(sign(X.dot(w)) != y)[0]
        
        FP, FN, prob = cal_prob(mistakes)
        if mis_pocket > FP*weight[0] + FN*weight[1]: 
            w_pocket = w
            mis_pocket = FP*weight[0] + FN*weight[1]
    return w_pocket, w

In [3]:
def get_err_rate(X, y, w):
    mistakes = np.where(sign(X.dot(w)) != y)[0]
    err_rate = len(mistakes) / len(X)
    return err_rate

In [4]:
def get_all_err_example(X, y, w):
    mistakes = np.where(sign(X.dot(w)) != y)[0]
    return np.hstack((X[mistakes],Y[mistakes].reshape(-1,1)))

In [5]:
data_train = read_data_from_file('hw1_18_train.dat') 
print('data_train shape: ', data_train.shape)
data_test = read_data_from_file('hw1_18_test.dat')
print('data_test shape: ', data_test.shape)
sign = np.frompyfunc(sign, 1, 1)

data_train shape:  (500, 5)
data_test shape:  (500, 5)


In [8]:
Y = data_train[:,-1]
X = np.concatenate((np.ones((data_train.shape[0],1)), data_train[:,:-1]), axis=1)

w_pocket, _ = weighted_pocket_pla(X,Y, weight=[1, 1])

In [9]:
errs = get_all_err_example(X, Y, w_pocket)

result = []
for i in [1, 5, 10, 50, 100, 1000]:
    w_pocket, _ = weighted_pocket_pla(X,Y, weight=[i*2, 1])
    errs = get_all_err_example(X, Y, w_pocket)
    result.append((len(np.where(errs[:,-1] < 0)[0]), len(np.where(errs[:,-1] > 0)[0])))
print(np.array(result))

[[ 14  38]
 [  7  68]
 [  0 186]
 [  0 176]
 [  0 188]
 [  0 185]]
